# Smart-G demo notebook , part 2</b>

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
from __future__ import absolute_import, division, print_function
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
from smartg.smartg import Smartg, Sensor
#from smartg.smartg import multi_profiles, reduce_diff
from smartg.smartg import RoughSurface, LambSurface, FlatSurface, Environment, Albedo_cst
from smartg.atmosphere import AtmAFGL, AeroOPAC, CloudOPAC, diff1, read_phase
from smartg.water import IOP_1, IOP, IOP_profile
from smartg.reptran import REPTRAN, reduce_reptran
from smartg.tools.tools import SpherIrr, Irr, reduce_Irr
from smartg.tools.luts import LUT, MLUT, Idx, merge, read_mlut
from smartg.tools.smartg_view import compare, plot_polar, spectrum , mdesc 
from smartg.tools.smartg_view import spectrum_view,transect_view,profile_view,phase_view,smartg_view,input_view
from ipywidgets import interact, interact_manual
# this module depends on the external HAPI Hitran Python interface
import hapi2018_gpu as hapi
from matplotlib import colors
colors_ = list(six.iteritems(colors.cnames))
import warnings
warnings.filterwarnings("ignore")
def get_spaced_colors(n):
    max_value = 16581375 #255**3
    interval = int(max_value / n)
    colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]
    return [(int(i[:2], 16)/256., int(i[2:4], 16)/256., int(i[4:], 16)/256.) for i in colors]

# Equivalence Theorem

In [ ]:
%%time
# Here we compare the TOA radiance simulated at 500 nm in the Chappuis band.
# Absorption is computed with Beer-Lambert Law (BEER=1) or layer single scattering albedo (BEER=0)

atm=AtmAFGL('afglt',grid=np.linspace(100.,0.,num=100), O3=300., NO2=False)
results = []
le={}
le.update(th=np.linspace(1.,85.,num=17)*np.pi/180)
le.update(phi=np.array([0., 90, 180, 270],dtype=np.float32)*np.pi/180)
S=Smartg(double=True, alt_pp=True)
for BEER in [0, 1]:
    results.append(S.run(THVDEG=45., wl=500., NBPHOTONS=1e7, le=le, atm=atm, BEER=BEER, stdev=True))

In [ ]:
# reference is Beer law
vmin = [0,  -0.15, -0.2, 0.]
vmax = [0.2, 0.05, 0.05, 100]
emax = [2e-4, 2e-4, 2e-4, .1]
ermax= [.2, .2, .2, .2]
_=compare(results[0], results[1], errb=True, vmin=vmin, vmax=vmax, 
      emax=emax, ermax=ermax, zenith_title='$VZA (^\circ)$')

# ALIS

## Atmosphere

### Pure scattering

In [ ]:
NWP= 1
S0= Smartg(alis=False, double=True, pp=True)
S = Smartg(alis=True , double=True, pp=True)

In [ ]:
%%time
dl  = 2.
lmin= 500.
lmax= 600.
wref= (lmax-lmin)/2 + lmin
NW  = int((lmax-lmin)/dl) + 1
wl  = np.linspace(lmin,lmax,num=NW)
NL  = 40
aer = AeroOPAC('urban',.3, 550.)
cld = CloudOPAC('wc.sol', 11., 9., 10., .3, 550.)
comp=[cld,aer]
znew =np.linspace(100.,0.,num=NL+1)
pfwav= np.linspace(lmin,lmax,num=NWP)
pfgrid = [100., 12., 7., 5., 3., 0.]
atm=AtmAFGL('afglus', grid=znew , O3=0., NO2=False, comp=comp, pfwav=pfwav, pfgrid=pfgrid)
pro=atm.calc(wl)

le={}
le.update(th=np.array([0.])*np.pi/180)
le.update(phi=np.array([90.])*np.pi/180)
NB=1e6

## !! nlow-1 has to divide NW-1 where NW is the number of wavelengths

mle1 = S.run( THVDEG=65., wl=wl, le=le,  NBPHOTONS=NB, atm=pro, progress=True, alis_options={'nlow':-1})
mle2 = S.run( THVDEG=65., wl=wl, le=le,  NBPHOTONS=NB, atm=pro, progress=True, alis_options={'nlow':-1})
m0le = S0.run(THVDEG=65., wl=wl, le=le,  NBPHOTONS=NB, atm=pro, progress=True)
print (mle2.attrs['kernel time (s)'])

In [ ]:
f=spectrum_view(mle1.sub({ 'Azimuth angles':0, 'Zenith angles':0}), fmt='-', color='r')
_=spectrum_view(mle2.sub({ 'Azimuth angles':0, 'Zenith angles':0}), fmt='-', color='g', fig=f)
_=spectrum_view(m0le.sub({ 'Azimuth angles':0, 'Zenith angles':0}), fmt='.', color='b', fig=f)#

## HITRAN

In [ ]:
hapi.db_begin('data')

### Spectrum and Jacobians

In [ ]:
%%time
# Sensitivity of O2A to surface pressure¶
# Just Rayleigh atmosphere

dl= 0.0200
lmin=759.
lmax=769.001
NW=int((lmax-lmin)/dl)
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW

hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
hapi.fetch('O2i3',7,3,vmin-100,vmax+100)

NL=21
print ('NW: ', NW)
znew =np.linspace(100.,0.,num=NL)

NB=1e7

P0l=[1000., 1010.]

ao2=np.zeros((NW,NL),dtype=float)

for i,P0 in enumerate(P0l):
    j=0
    atm = AtmAFGL('afglmw', grid=znew, NO2=False, O3=0., P0=P0)
    
    for p,t,o2,z in zip(atm.prof.P,atm.prof.T,atm.prof.dens_o2,atm.prof.z):
        nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=True,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
        ao2[:,j] = coefo2 * o2 * 1e5
        j=j+1
    wl=(1e7/nuo2)
    wl=wl[::1]

    # We concatenate both atmospheres (perturbed and une perturbed) along the 'wavelength' dimension
    pro = AtmAFGL('afglmw', grid=znew, NO2=False, O3=0., P0=P0).calc(wl)
    if i!=0 :      
        ab  = np.concatenate((ab , ao2), axis=0)
        ray = np.concatenate((ray, pro['OD_r'].data), axis=0)
        
    else: 
        ab = ao2
        ray= pro['OD_r'].data

wls = np.concatenate((wl,wl))
print (ao2.max())
atm = AtmAFGL('afglmw', grid=znew, prof_ray=ray, prof_abs=ab)

In [ ]:
S0=Smartg(alis=True, double=True, pp=True)
tv=np.linspace(0.,75.,num=6)
le={}
le.update(th=tv*np.pi/180)
le.update(phi=np.linspace(0.,360.,num=1)*np.pi/180)
#surf=RoughSurface(SUR=1)
surf=None

In [ ]:
%%time
# Compute spectrum with 3 low resolution wavelength for scattering correctiuons, one jacobian
alis_options={'nlow':3, 'njac':1}
m=S0.run(THVDEG=30., wl=wls, atm=atm, surf=surf, le=le, NBPHOTONS=1e5, 
         progress=False, alis_options=alis_options)

In [ ]:
field='I_up (TOA)'
I=m[field].sub(d={'wavelength':arange(NW)})
I.names[0]=r'$\lambda (nm)$'
I.axes[0]=wl
I.desc=mdesc(field)
for t in tv[:-1]:
    _= I.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.04, vmin=0, label='%.0f'%t)
legend()

figure()
jac= (m[field].sub(d={'wavelength':arange(NW)+NW}) - 
      m[field].sub(d={'wavelength':arange(NW)}) ) / (P0l[-1]-P0l[0]) 
jac.names[0]=r'$\lambda (nm)$'
jac.axes[0]=wl
jac.desc=r'$d$'+mdesc(field)+r'$/dP_0$' + '  ' + r'$(hPa^{-1})$'

for t in tv[:-1]:
    _=jac.sub()[:,0,Idx(t)].plot(fmt='-', vmax=2.e-4, vmin=0, label='%.0f'%t)
legend()

figure()
field='Q_up (TOA)'
I=m[field].sub(d={'wavelength':arange(NW)})
I.names[0]=r'$\lambda (nm)$'
I.axes[0]=wl
I.desc=mdesc(field)
for t in tv[:-1]:
    _= I.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.001, vmin=-0.005, label='%.0f'%t)
#legend()

figure()
jac= (m[field].sub(d={'wavelength':arange(NW)+NW}) - 
      m[field].sub(d={'wavelength':arange(NW)}) ) / (P0l[-1]-P0l[0]) 
jac.names[0]=r'$\lambda (nm)$'
jac.axes[0]=wl
jac.desc=r'$d$'+mdesc(field)+r'$/dP_0$' + '  ' + r'$(hPa^{-1})$'

for t in tv[:-1]:
    _=jac.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.5e-5, vmin=-3.e-5, label='%.0f'%t)
#legend(loc='best')

## Validation against ART-DECO

Doubling Adding; 32 streams

### Rayleigh  and black surface

In [ ]:
S=Smartg(alis=True, double=True, pp=True, alt_pp=True)

In [ ]:
fgas = '../smartg/validation/cTauGas_ray_O2.dat'
gas_valid = diff1(np.loadtxt(fgas, skiprows=7)[:,1:].T, axis=1)
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '../smartg/validation/cTauRay_ray_O2.dat'
ray_valid = diff1(np.loadtxt(fray, skiprows=7)[:,1:].T, axis=1)
data_valid=np.loadtxt('../smartg/validation/artdeco_lbl_nstr_32_ray_O2.dat')

In [ ]:
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.], 
                    prof_ray= ray_valid, prof_abs= gas_valid)

In [ ]:
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
mle_valid5=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e9, atm=atm_valid, 
               NBLOOP=1e5, progress=True, alis_options={'nlow':3})
print (mle_valid5.attrs['kernel time (s)'])

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid5['Q_up (TOA)']*mle_valid5['Q_up (TOA)']+mle_valid5['U_up (TOA)']*mle_valid5['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))
i=mle_valid5['I_up (TOA)']
##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.01)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=1, vmin=-1)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.005)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=1, vmin=-1)

### Rayleigh , aerosol and black surface

In [ ]:
S=Smartg(alis=True, double=True, pp=True, alt_pp=True)

In [ ]:
#typ='desert'
typ='urban' # tau=0.25
####################""""""
fgas = '../smartg/validation/cTauGas_ray_%s_O2.dat'%typ
gas_valid = diff1(np.loadtxt(fgas, skiprows=7)[:,1:].T, axis=1)
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '../smartg/validation/cTauRay_ray_%s_O2.dat'%typ
ray_valid = diff1(np.loadtxt(fray, skiprows=7)[:,1:].T, axis=1)
faer_abs = '../smartg/validation/cTauAbs_ptcle_ray_%s_O2.dat'%typ
aer_abs_valid = diff1(np.loadtxt(faer_abs, skiprows=7)[:,1:].T, axis=1)
faer_sca = '../smartg/validation/cTauSca_ptcle_ray_%s_O2.dat'%typ
aer_sca_valid = diff1(np.loadtxt(faer_sca, skiprows=7)[:,1:].T, axis=1)

# aerosols phase matrix import
faer_phase= '../smartg/validation/phasemat_ray_%s_O2.dat'%typ
f=open(faer_phase,'r')
N=np.genfromtxt(faer_phase, usecols=range(1), max_rows=1, dtype=int)
pfwav=[]
P=[]
Npf=3
data=np.zeros((Npf,1,N,5), dtype=float32) 
for k in range(Npf):
    pfwav.append(np.genfromtxt(faer_phase, usecols=range(1), skip_header=(1+(2+N)*k), max_rows=1))
    pizero=np.genfromtxt(faer_phase, usecols=range(1), skip_header=(1+(2+N)*k+1), max_rows=1)
    data[k,0,:,:] = np.genfromtxt(faer_phase, usecols=range(5), skip_header=(1+(2+N)*k+2), max_rows=N)
    
data=data.swapaxes(2,3)
phase_valid = LUT(data[:,:,1:,:],
          names = ['wav_phase_atm', 'z_phase_atm', 'stk','theta'] ,
          axes  = [pfwav, [0], None, data[0,0,0,:]])
   
data_valid=np.loadtxt('../smartg/validation/artdeco_lbl_nstr_32_ray_%s_O2.dat'%typ)

In [ ]:
aer_ext_valid  = aer_sca_valid + aer_abs_valid
aer_ssa_valid  = aer_sca_valid / aer_ext_valid
aer_ssa_valid[aer_ext_valid==0]=1.

comp=[AeroOPAC('desert',0.5, 550., phase= phase_valid)]
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=pfwav, comp=comp,
                    prof_ray= ray_valid,
                    prof_aer= (aer_ext_valid,aer_ssa_valid),
                    prof_abs= gas_valid
                    )

In [ ]:
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
mle_valid5=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e10, atm=atm_valid, 
                   NBLOOP=1e5, progress=True, alis_options={'nlow':9})
print (mle_valid5.attrs['kernel time (s)'])

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid5['Q_up (TOA)']*mle_valid5['Q_up (TOA)']+mle_valid5['U_up (TOA)']*mle_valid5['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))
i=mle_valid5['I_up (TOA)']
##
figure(figsize=(8,4))
plot(w_valid,  i_valid, 'r-')
f= i.sub()[:,0,0].plot(fmt='g--', vmax=0.02)
figure(figsize=(8,4))
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=1, vmin=-1)
##
figure()
plot(w_valid,  ip_valid, 'r-')
f= ip.sub()[:,0,0].plot(fmt='g--', vmax=0.006)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=1, vmin=-1)

## With water

In [ ]:
# alt_pp=True is mandatory
NWP= 1
S0= Smartg(alis=None , double=True, pp=True, alt_pp=True)
S = Smartg(alis=True , double=True, pp=True, alt_pp=True)

dl  = 2
lmin= 500.
lmax= 550.
wref= (lmax-lmin)/2 + lmin
NW  = int((lmax-lmin)/dl) + 1
wl  = np.linspace(lmin,lmax,num=NW)
NL  = 40
pfwav= np.linspace(lmin,lmax,num=NWP)
atm=AtmAFGL('afglus', pfwav=pfwav)
water = IOP_1(0.3, pfwav=pfwav, DEPTH=10)
surf  = RoughSurface(WIND=12.)
le={}
le.update(th=np.array([45.])*np.pi/180)
le.update(phi=np.array([90.])*np.pi/180)

In [ ]:
%%time
alis={'nlow':-1, 'njac':0}
NB=1e7

mle1 = S.run( THVDEG=65., wl=wl, le=le,  NBPHOTONS=NB, atm=atm, surf=surf, water=water, 
             OUTPUT_LAYERS=3, alis_options=alis)
mle2 = S.run( THVDEG=65., wl=wl, le=le,  NBPHOTONS=NB, atm=atm, surf=surf, water=water, 
             OUTPUT_LAYERS=3, alis_options=alis)
m0le = S0.run(THVDEG=65., wl=wl, le=le,  NBPHOTONS=NB, atm=atm, surf=surf, water=water, 
              OUTPUT_LAYERS=3)
print (mle2.attrs['kernel time (s)'])

In [ ]:
f=spectrum_view(mle1.sub({ 'Azimuth angles':0, 'Zenith angles':0}), fmt='-', color='r', field='up (0-)')
_=spectrum_view(mle2.sub({ 'Azimuth angles':0, 'Zenith angles':0}), fmt='-', color='g', field='up (0-)', fig=f)
_=spectrum_view(m0le.sub({ 'Azimuth angles':0, 'Zenith angles':0}), fmt='.', color='b', field='up (0-)', fig=f)#

### Spectrum and Jacobians

In [ ]:
# Ozone and Chlorophyll sensitivities
# 1 profile + 2 perturbed profiles (2 jacobians) = 3 profiles
atms = []
O3_base = 200.
DO3 = 20.
waters = []
Chl_base = 0.3
DChl = 0.03

for O3 in [O3_base, O3_base + DO3, O3_base]: # 2nd profile is O3 perturbation
    atms.append(AtmAFGL('afglt', O3=O3, grid='120[20]0').calc(wl))

for Chl in [Chl_base, Chl_base, Chl_base + DChl]: # 3rd profile is Chl perturbation                                                          
    waters.append(IOP_profile(Chl, NLAYER=11, DEPTH=100, ang_trunc=5, pfwav = pfwav, NANG=7201).calc(wl))
    #waters.append(IOP_1(Chl, DEPTH=10, pfwav = pfwav, NANG=7201).calc(wl))
    
# wavelengths is the repetition of wl  for 3 profiles
wls   = np.concatenate((wl,wl,wl))
# construction of multi_profiles from profiles list, to be ingested by smartg.run() method
water = multi_profiles(waters, kind='oc')
atm   = multi_profiles(atms,   kind='atm')

In [ ]:
%%time
tv=np.linspace(0., 75.,num=1)
phi=np.linspace(0.,360,num=1)
le={'th_deg':tv, 'phi_deg':phi}
alis_options={'nlow':-1, 'njac':2}
# two jacobians here
m=S.run(THVDEG=65., wl=wls, atm=atm, surf=surf, water=water, le=le, OUTPUT_LAYERS=3,
         NBPHOTONS=1e5, alis_options=alis_options)

In [ ]:
Jacobians = reduce_diff(m, ['O3', 'Chl'], delta=[DO3, DChl]).dropaxis('Azimuth angles','Zenith angles')
#Jacobians.describe()
m.describe()

In [ ]:
fig, ax= subplots(3)
fig.set_size_inches(8,10)

for (i,field) in enumerate(['I_up (0-)', 'dI_up (0-)/dO3', 'dI_up (0-)/dChl']):
    sca(ax[i])
    l = Jacobians[field]
    l.desc = mdesc(field)
    _= l.plot(fmt='.-')

In [ ]:
m.dropaxis('Azimuth angles','Zenith angles')['cdist_up (0-)'].data[:4]\
/m.dropaxis('Azimuth angles','Zenith angles')['N_up (0-)'].data[:4]

In [ ]:
#m.sub(d={'wavelength':arange(NW), 'Azimuth angles':0, 'Zenith angles':0}).describe()
f=spectrum_view(m.sub(d={'wavelength':arange(NW)+2*NW, 'Azimuth angles':1, 'Zenith angles':0}),
               field='up (0-)', QU=True)
for (i,t) in enumerate(tv[1:-1]):
    _=spectrum_view(m.sub(d={'wavelength':arange(NW)+2*NW, 'Azimuth angles':1, 'Zenith angles':i+1}),
               field='up (0-)', fig=f, QU=True)

In [ ]:
_=smartg_view(Jacobians.sub(d={'wavelength':Idx(530)}), field='up (0-)', 
              prefix='')

_=smartg_view(Jacobians.sub(d={'wavelength':Idx(530)}), field='up (0-)/dChl', 
              prefix='d')

In [ ]:
f=spectrum_view(Jacobians.sub(d={'Zenith angles':Idx(45),'Azimuth angles':0}), field='up (0-)', 
              prefix='')
df=spectrum_view(Jacobians.sub(d={'Zenith angles':Idx(45),'Azimuth angles':0}), field='up (0-)/dChl', 
              prefix='d')
for k in range(5):
    _=spectrum_view(Jacobians.sub(d={'Zenith angles':Idx(45),'Azimuth angles':k+1}), field='up (0-)', 
              prefix='', fig=f)
    _=spectrum_view(Jacobians.sub(d={'Zenith angles':Idx(45),'Azimuth angles':k+1}), field='up (0-)/dChl', 
              prefix='d', fig=df)